# Y4
## Javier Jorganes
## 9.12.2023

In [1]:
#!pip install moviepy
#!pip install --upgrade spyder
#!pip install opencv-python
#!pip install ipywidgets
#!pip install ipympl
#!pip install hvplot holoviews

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import ipywidgets as widgets
from orientation import *
%matplotlib notebook

import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

import cv2
import ipywidgets as widgets
from IPython.display import display, clear_output, Image
from PIL import Image

In [2]:
plt.rcParams['figure.figsize'] = (12, 5)

In [3]:

df = pd.read_csv('v5_2.csv')

time, gx, gy, gz, wx, wy, wz = get_components(df) 

theta, phi = calc_angles(time, gy, gx, gz, wy, wx, wz)


## IDEA
Luulen, että tämä oli liian suuri haaste siihen aikaan nähden, jonka luulin aluksi tarvitsevani.
Ideana on, että video synkronoidaan kuvaajan kanssa. Kun liikutat jotakin liukusäädintä, video liikkuu ja kuvaaja osoittaa automaattisesti 4 sekunnin alueen pysäytetyn kuvan ympärillä videossa. Molemmilla on sama kesto ja ne on synkronoitu.

Videota on editoitu kahdella punaisella viivalla, jotka simuloivat linjalasereita.

Ajanpuutteen vuoksi se ei näytä "niin kivalta" kuin olisin halunnut. Olin myös suunnitellut lisääväni muita tärkeitä tietoja, kuten keskimääräisen kulman arvo kyseisellä aikavyöhykkeellä, mutta en ehtinyt toteuttaa sitä.

In [4]:
video_path = 'k1.mp4' 

cap = cv2.VideoCapture(video_path)

# Get video properties
max_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
max_time = max_frames / fps

current_time = 0

def display_frame(second):
    global current_time
    current_time = second  # Update the global variable with the current slider value
    cap.set(cv2.CAP_PROP_POS_MSEC, second*1000)  # Set the time in milliseconds
    ret, frame = cap.read()
    if ret:
        # Convert the image color to RGB and then to an Image object
        img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        # Display the image
        display(img)
    else:
        print("Error: Unable to capture video frame.")

slider = widgets.FloatSlider(
    min=0,
    max=int(max_time),
    step=0.01,  # Step size of 0.01 second
    description='Time (s):'
)

widgets.interactive(display_frame, second=slider)



interactive(children=(FloatSlider(value=0.0, description='Time (s):', max=20.0, step=0.01), Output()), _dom_cl…

In [5]:
# Create an output widget to display the current time
output = widgets.Output()

def display_frame(second):
    global current_time
    current_time = second
    with output:
        clear_output(wait=True)  # Clear the previous output
        df = pd.DataFrame({'time': time, 'theta': theta, 'phi': phi})
        start_time = max(current_time - 2, 0)  # Ensure start_time is not negative
        end_time = min(current_time + 2, df['time'].max())  # Ensure end_time does not exceed data range
        # Plot for phi
        phi_plot = df.hvplot.line('time', 'phi', color='green', title='Phi Angle', ylabel='Phi Angle ($\degree$)')
        phi_plot = phi_plot.opts(xlim=(start_time, end_time))
        display(phi_plot)  # Explicitly display the plot
        theta_plot = df.hvplot.line('time', 'theta', color='red', title='theta Angle', ylabel='theta Angle ($\degree$)')
        theta_plot = theta_plot.opts(xlim=(start_time, end_time))
        display(theta_plot)  # Explicitly display the plot

# ... rest of your existing display_frame function ...

# Display the output widget
display(output)

# Display the slider
widgets.interactive(display_frame, second=slider)

Output()

interactive(children=(FloatSlider(value=0.0, description='Time (s):', max=20.0, step=0.01), Output()), _dom_cl…

## koko kuvaajan

In [6]:
# Plot for theta
df = pd.DataFrame({'time': time, 'theta': theta,'phi': phi})
theta_plot = df.hvplot.line('time', 'theta', color='red', title='Theta Angle', ylabel='Theta Angle ($\degree$)')
theta_plot


:Curve   [time]   (theta)

In [7]:
# Plot for phi

phi_plot = df.hvplot.line('time', 'phi', color='green', title='phi Angle', ylabel='phi Angle ($\degree$)')
phi_plot


:Curve   [time]   (phi)